In [1]:
import pandas as pd
import numpy as np

# IEEE-ICCE-RL-JSP

In this paper, authors use Graph Neural Network to determine which heuristics to choose. Heuristics considered in the following experiment are FIFO, MOR, SPT, and MWKR.

## Model

I had to train my own models using the script ['train_dqn.py'](train_dqn.py) provided by the owner of the repository. Models were trained on random instances with size up to 10x10. To check if my trained models are valid, I compared the experimental result presented in the paper with my results on the same instances. The results from the paper are

|        | IEEE-ICCE-RL-JSP  | ScheduleNet | L2D   | MOR   | FIFO  | SPT   | MWKR  |
| ------ | ----- |:----------- |:----- |:----- |:----- |:----- |:----- |
| Gap(S) | 20.0% | 17.7%       | 30.7% | 22.7% | 29.0% | 31.2% | 22.7% |
| Gap(L) | 10.5% | 11.3%       | 20.8% | 14.8% | 19.4% | 21.3% | 14.5% |
| Time   | 1.55s | N/A         | 5.10s | 1.18s | 1.12s | 1.01s | 1.13s |

Results of my 3 trained models are (on average)

In [2]:
df = pd.read_csv('eval_dqn.csv')
large = df[(df['J'] >= 50) & (df['M'] >= 15)]
small = df[~(df['J'] >= 50) & (df['M'] >= 15)]

print(f"Average time is {-df['Time'].mean().round(2)}")
print(f"Average gap of small instances is {small['Gap'].mean().round(4) * 100}%")
print(f"Average gap of large instances is {large['Gap'].mean().round(4) * 100}%")

Average time is 11.87
Average gap of small instances is 20.89%
Average gap of large instances is 11.44%


The time is much higher, because I used `cpu` instead of `cuda`. Average gaps approximately match the results of the paper, so I decided my trained model is valid.

## Preprocessing

This model processes instances given in [Standard specifiation](http://jobshop.jjvh.nl/explanation.php#standard_def). I have our benchmarks also in Standard Specification, but for the sake of being consistent, I will consider only instances in [Taillard specification](http://jobshop.jjvh.nl/explanation.php#taillard_def). Therefore I have to write a function, which converts problem in taillard specification to standard specification.

In [3]:
def parse_instance_taillard(filename):
    '''Parses instance written in Taillard specification: http://jobshop.jjvh.nl/explanation.php
    
      Args:
        filename - file containing the instance in Taillard specification

      Returns:
        number of jobs,
        number of machines,
        the processor times for each operation,
        the order for visiting the machines
    '''

    with open(filename, 'r') as f:
        # parse number of jobs J and machines M
        J, M = map(int, f.readline().split())

        # Initialize two empty numpy arrays with dimensions J x M
        processor_times = np.empty((J, M), dtype=int)
        orders_of_machines = np.empty((J, M), dtype=int)
    
        # Read the next J lines containing processor times
        for i in range(J):
            processor_times[i] = list(map(int, f.readline().split()))
    
        # Read the next J lines containing orders of machines
        for i in range(J):
            orders_of_machines[i] = list(map(int, f.readline().split()))

        return J, M, processor_times, orders_of_machines

def taillard_to_standard(taillard_instance):
    # parse taillard instance
    J, M, processor_times, orders_of_machines = parse_instance_taillard(taillard_instance)

    # save as standard instance
    standard_instance = "/tmp/standard_" + taillard_instance.split("/")[-1]
    with open(standard_instance, 'w') as f:
        # save number of jobs and machines
        f.write(f"{J}\t{M}\n")

        for job in range(J):
            for machine in range(M):
                f.write(f'{orders_of_machines[job][machine] - 1}\t{processor_times[job][machine]}\t')

            f.write('\n')

    return standard_instance

## Experiment on our benchmarks

In [5]:
# import some stuff and define helper functions
import os, argparse
from eval_dqn import eval_dqn

def get_all_instances_in_taillard_specification():
    '''Lists all instances in Taillard specification'''
    matching_files = []
    root_dir = "../../../benchmarks/jssp/"
    target_string = "Taillard_specification"

    for foldername, subfolders, filenames in os.walk(root_dir):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            if target_string in filepath:
                matching_files.append(filepath)

    return matching_files

# need command line arguments for the model
parser = argparse.ArgumentParser(description=__doc__)
parser.add_argument('-d', '--device', default='cpu')
# arguments for DQN
parser.add_argument('--warmup', default=10000, type=int)
parser.add_argument('--episode', default=100000, type=int)
parser.add_argument('--capacity', default=10000, type=int)
parser.add_argument('--batch_size', default=32, type=int)
parser.add_argument('--lr', default=.01, type=float)
parser.add_argument('--eps', default=0.0, type=float)
parser.add_argument('--eps_decay', default=.995, type=float)
parser.add_argument('--eps_min', default=.01, type=float)
parser.add_argument('--gamma', default=1.0, type=float)
parser.add_argument('--freq', default=4, type=int)
parser.add_argument('--target_freq', default=1000, type=int)
parser.add_argument('--double', action='store_true')
parser.add_argument(
    '--max_process_time',
    type=int,
    default=100,
    help='Maximum Process Time of an Operation')
args = parser.parse_args(args=[])

# load models and instances
MODELS_PATH = 'agent/DQN/weight'
models = os.listdir(MODELS_PATH)
instances = sorted(get_all_instances_in_taillard_specification())

# run experiments
for model in models:
    for instance in instances:
        model_path = os.path.join(MODELS_PATH, model)
        makespan, run_time = eval_dqn(model_path, taillard_to_standard(instance), args)
        print(f"Model: {model}, instance: {instance.split('/')[-1]}, makespan: {makespan}, time: {np.round(run_time, 2)}")

Model: DQN_ep1670, instance: abz5.txt, makespan: 1369, time: 0.41
Model: DQN_ep1670, instance: abz6.txt, makespan: 1011, time: 0.39
Model: DQN_ep1670, instance: abz7.txt, makespan: 767, time: 1.6
Model: DQN_ep1670, instance: abz8.txt, makespan: 773, time: 1.7
Model: DQN_ep1670, instance: abz9.txt, makespan: 853, time: 1.67
Model: DQN_ep1670, instance: dmu01.txt, makespan: 3273, time: 1.57
Model: DQN_ep1670, instance: dmu02.txt, makespan: 3331, time: 1.55
Model: DQN_ep1670, instance: dmu03.txt, makespan: 3272, time: 1.6
Model: DQN_ep1670, instance: dmu04.txt, makespan: 3087, time: 1.57
Model: DQN_ep1670, instance: dmu05.txt, makespan: 3160, time: 1.61
Model: DQN_ep1670, instance: dmu06.txt, makespan: 3944, time: 2.09
Model: DQN_ep1670, instance: dmu07.txt, makespan: 3635, time: 2.19
Model: DQN_ep1670, instance: dmu08.txt, makespan: 3858, time: 2.21
Model: DQN_ep1670, instance: dmu09.txt, makespan: 4212, time: 2.08
Model: DQN_ep1670, instance: dmu10.txt, makespan: 3505, time: 1.99
Model: